## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


d:\program files\python\python37\tutorial-env\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
d:\program files\python\python37\tutorial-env\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [2]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


Exception in callback Transaction.__retry()
handle: <TimerHandle when=70399.046 Transaction.__retry()>
Traceback (most recent call last):
  File "D:\program files\python\python37\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "d:\program files\python\python37\tutorial-env\lib\site-packages\aioice\stun.py", line 306, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "D:\program files\python\python37\lib\asyncio\futures.py", line 246, in set_exception
    raise InvalidStateError('{}: {!r}'.format(self._state, self))
asyncio.base_futures.InvalidStateError: FINISHED: <Future finished result=(Message(messa...\x17\x8e\xf6'), ('172.31.48.1', 56799))>


In [3]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: bed3e121100a4738bd415caac7bb0e85>,[DO1 data],"Dataset of 6 samples, 1 feature",<class 'torch.Tensor'>


In [4]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 41cfbabb65f14f029c195045770a1b07>,[DO2 data],"Dataset of 5 samples, 1 feature",<class 'torch.Tensor'>


## Linear regression

In [5]:
data1_ptr = duet1.store[0]
data2_ptr = duet2.store[0]

print(data1_ptr)
print(data2_ptr)

### Create Base Model

In [6]:
import torch

In [7]:
in_dim = 1
out_dim = 1

In [8]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

### Training

In [9]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        loss = torch_ref.nn.functional.mse_loss(output, target_ptr)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses

#### Send one copy of the model to each data owner or client and train them remotely one by one

In [10]:
import torch as th
import numpy as np

In [11]:
base_model = SyNet(torch)

Train on Data Owner 1 data

In [12]:
remote_model1 = base_model.send(duet1)

In [13]:
remote_torch1 = duet1.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

Dummy target data

In [14]:
target1_ptr = th.FloatTensor(np.array([5, 10, 15, 22, 30, 38]).reshape(-1, 1))
target1_ptr

tensor([[ 5.],
        [10.],
        [15.],
        [22.],
        [30.],
        [38.]])

In [15]:
iteration = 10
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

Epoch 0 loss 50.621063232421875
Epoch 1 loss 13.572895050048828
Epoch 2 loss 1.601438045501709
Epoch 3 loss 7.448986053466797
Epoch 4 loss 16.346715927124023
Epoch 5 loss 18.442180633544922
Epoch 6 loss 14.163008689880371
Epoch 7 loss 7.744306564331055
Epoch 8 loss 2.935067892074585
Epoch 9 loss 1.5995491743087769


Train on Data Owner 2 data

In [16]:
remote_model2 = base_model.send(duet2)

In [17]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

Dummy Target data

In [18]:
target2_ptr = th.FloatTensor(np.array([35, 40, 45, 55, 60]).reshape(-1, 1))
target2_ptr

tensor([[35.],
        [40.],
        [45.],
        [55.],
        [60.]])

In [19]:
iteration = 10
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 138.23800659179688
Epoch 1 loss 17.31894302368164
Epoch 2 loss 11.224181175231934
Epoch 3 loss 49.8996467590332
Epoch 4 loss 59.34345626831055
Epoch 5 loss 39.10646438598633
Epoch 6 loss 14.231210708618164
Epoch 7 loss 2.707404375076294
Epoch 8 loss 8.063619613647461
Epoch 9 loss 20.713214874267578


### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [20]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[0.4773]], requires_grad=True), Parameter containing:
tensor([-0.5709], requires_grad=True)]

Remote model1 parameters:
[Parameter containing:
tensor([[0.6364]], requires_grad=True), Parameter containing:
tensor([-0.3874], requires_grad=True)]

Remote model2 parameters:
[Parameter containing:
tensor([[0.5623]], requires_grad=True), Parameter containing:
tensor([-0.5091], requires_grad=True)]


As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [21]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

OrderedDict([('linear.weight', tensor([[0.6364]])), ('linear.bias', tensor([-0.3874]))])


In [22]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('linear.weight', tensor([[0.5623]])), ('linear.bias', tensor([-0.5091]))])


In [23]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [24]:
avg_updates = OrderedDict()
avg_updates["linear.weight"] = (
    remote_model1_updates["linear.weight"] + remote_model2_updates["linear.weight"]
) / 2
avg_updates["linear.bias"] = (
    remote_model1_updates["linear.bias"] + remote_model2_updates["linear.bias"]
) / 2

print(avg_updates)

OrderedDict([('linear.weight', tensor([[0.5993]])), ('linear.bias', tensor([-0.4483]))])


### Load aggregated weights

In [25]:
combined_model = SyNet(torch)

In [26]:
combined_model.load_state_dict(avg_updates)

In [27]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [28]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [29]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 9.740633964538574 Ground Truth: 12.0
Prediction: 14.535414695739746 Ground Truth: 15.0
Prediction: 18.73084831237793 Ground Truth: 20.0
Prediction: 29.519105911254883 Ground Truth: 30.0
Prediction: 47.49953079223633 Ground Truth: 50.0


In [30]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 7.543947219848633 Ground Truth: 12.0
Prediction: 11.362720489501953 Ground Truth: 15.0
Prediction: 14.704147338867188 Ground Truth: 20.0
Prediction: 23.29638671875 Ground Truth: 30.0
Prediction: 37.61678695678711 Ground Truth: 50.0


## Comparison to classical linear regression on centralised data

In [31]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [32]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [33]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [34]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [35]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

Epoch 0 loss 4104.67041015625
Epoch 10 loss 85.96489715576172
Epoch 20 loss 267.0783386230469
Epoch 30 loss 33.182125091552734
Epoch 40 loss 34.94289016723633
Epoch 50 loss 5.904316425323486
Epoch 60 loss 8.275069236755371
Epoch 70 loss 2.9466781616210938
Epoch 80 loss 3.6290879249572754
Epoch 90 loss 3.051913261413574


In [36]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [37]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 11.170931816101074 Ground Truth: 12.0
Prediction: 16.158401489257812 Ground Truth: 15.0
Prediction: 20.522436141967773 Ground Truth: 20.0
Prediction: 31.744239807128906 Ground Truth: 30.0
Prediction: 50.44724655151367 Ground Truth: 50.0
